In [26]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import gensim
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from bertopic import BERTopic

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import pipeline

from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Mounted at /content/drive


In [9]:
df = pd.read_csv('/content/drive/MyDrive/shohei-cleaned-gptig.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Comparing Sentiment

### Vader

In [10]:
# Initialize VADER Sentiment Analyzer
vader_analyzer = SentimentIntensityAnalyzer()
df["vaderSent"] = df['en_text'].apply(lambda x: vader_analyzer.polarity_scores(x))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
df["vaderSent"]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,vaderSent
0,"{'neg': 0.0, 'neu': 0.816, 'pos': 0.184, 'comp..."
1,"{'neg': 0.028, 'neu': 0.904, 'pos': 0.068, 'co..."
2,"{'neg': 0.0, 'neu': 0.928, 'pos': 0.072, 'comp..."
3,"{'neg': 0.0, 'neu': 0.931, 'pos': 0.069, 'comp..."
4,"{'neg': 0.03, 'neu': 0.914, 'pos': 0.056, 'com..."
...,...
23239,"{'neg': 0.03, 'neu': 0.97, 'pos': 0.0, 'compou..."
23240,"{'neg': 0.062, 'neu': 0.82, 'pos': 0.117, 'com..."
23241,"{'neg': 0.024, 'neu': 0.663, 'pos': 0.312, 'co..."
23242,"{'neg': 0.042, 'neu': 0.8, 'pos': 0.158, 'comp..."


### BERT

In [12]:
df["trunc_text"] = df['en_text'].apply(lambda x: x[:500])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
import torch


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
bert_classifier = pipeline('sentiment-analysis',)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [18]:
bert_scores = bert_classifier(df.trunc_text.tolist())

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Token indices sequence length is longer than the specified maximum sequence length for this model (528 > 512). Running this sequence through the model will result in indexing errors


RuntimeError: The size of tensor a (528) must match the size of tensor b (512) at non-singleton dimension 1

In [ ]:
df["bertSent"] =  bert_scores


NameError: name 'bert_scores' is not defined

# Topic Modeling

### Latent Dirichlet Allocation

In [19]:
num_topics = 10

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
import nltk
nltk.download('stopwords')


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [23]:
list_stopwords = stopwords.words("english")
porter = PorterStemmer()
def process_step(one_str):
    nostop_listing = [word for word in wordpunct_tokenize(one_str)
                      if word not in list_stopwords]
    clean_listing = [porter.stem(word) for word in nostop_listing
                    if word.isalpha()
                    and len(word) > 3]
    clean_listing_str = " ".join(clean_listing)
    return(clean_listing_str)

df["text_proc"] = df['en_text'].apply(process_step)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [139]:
df_jp = df[(df['lang'] == 'ja') ]


df_kr = df[(df['lang'] == 'ko') ]

df_zh = df[(df['lang'] == 'zh') ]


df_es = df[(df['lang'] == 'es') ]


df_jpam = df[(df['is_jpam'] == 1) ]

df_koram = df[(df['is_koram'] == 1) ]

df_zham = df[(df['is_cham'] == 1) ]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [130]:
df_temp = df[
           (df['is_jpam'] == 1) ]

          #  | (df['is_koram'] == 1) |
          #   (df['is_cham'] == 1) | (df['is_aam'] == 1)]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [133]:
# Preprocess data
documents_clean = [gensim.utils.simple_preprocess(doc) for doc in df_temp.text_proc]
dictionary = corpora.Dictionary(documents_clean)
corpus = [dictionary.doc2bow(doc) for doc in documents_clean]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [134]:
lda_model = LdaModel(corpus=corpus, num_topics=10, id2word=dictionary, random_state=42)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [135]:
topics = lda_model.print_topics(num_words = 10)

for topic in topics:
    print(topic)


(0, '0.052*"shohei" + 0.037*"dodger" + 0.024*"shoheiohtani" + 0.021*"award" + 0.019*"today" + 0.016*"ohtani" + 0.013*"player" + 0.013*"stadium" + 0.013*"flower" + 0.010*"angel"')
(1, '0.018*"hanshintig" + 0.017*"radio" + 0.007*"funni" + 0.007*"small" + 0.006*"approxim" + 0.005*"shihtzu" + 0.004*"alon" + 0.004*"home" + 0.004*"messag" + 0.003*"dodger"')
(2, '0.077*"dodger" + 0.043*"shoheiohtani" + 0.027*"大谷翔平" + 0.017*"shohei" + 0.013*"losangelesdodg" + 0.011*"basebal" + 0.011*"ohtani" + 0.011*"repost" + 0.010*"오타니쇼헤이" + 0.008*"blue"')
(3, '0.029*"shohei" + 0.028*"shoheiohtani" + 0.021*"ohtani" + 0.014*"dodger" + 0.013*"spring" + 0.013*"game" + 0.011*"good" + 0.011*"look" + 0.010*"train" + 0.010*"also"')
(4, '0.028*"ohtani" + 0.025*"shohei" + 0.019*"shoheiohtani" + 0.018*"unanim" + 0.017*"leagu" + 0.012*"time" + 0.012*"player" + 0.012*"home" + 0.011*"showtim" + 0.010*"base"')
(5, '0.039*"ippei" + 0.017*"shoheiohtani" + 0.012*"billion" + 0.011*"http" + 0.011*"ohtani" + 0.010*"princ" + 0.0

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [136]:
### visualize
lda_display = gensimvis.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(lda_display)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## BERTopic

In [141]:
groups = [df_jp, df_kr, df_zh , df_es, df_jpam , df_koram , df_zham]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [153]:
%%time
topic_model = BERTopic()
topics, probs = topic_model.fit_transform(groups[6].en_text)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


CPU times: user 483 ms, sys: 21.9 ms, total: 505 ms
Wall time: 1.28 s


In [154]:
topic_model.get_topic_info()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Topic,Count,Name,Representation,Representative_Docs
0,-1,40,-1_the_and_of_to,"[the, and, of, to, in, ohtani, redactedmention...","[Ohtani is beginning a record 10-year, $700 mi..."
